# Experiment records
- Trained model name: resnet_trained_model_digit_with_printed_v1_new_finetune_13_08_22_epoch_5.h5 (Fine-tuned on base model)
- Trained for: 22 epochs
- Augmentations used: No
- Accuracy achieved: 66.49%
- Dataset used: (NIST dataset) - https://s3.amazonaws.com/nist-srd/SD19/by_class.zip
- Description of experiment:To check the performance of above model on selected grayscale images of NSIT dataset

In [1]:
import tensorflow as tf
import sys  
import os
sys.path.append(os.path.realpath('../../src'))
from predict import pred_using_h5_digit, pred_using_tflite_model
import cv2
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

2022-12-23 17:27:58.464389: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-23 17:27:58.464410: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = tf.keras.models.load_model('../../models/pre-trained_model/resnet_trained_model_digit_with_printed_v1_new_finetune_13_08_22_epoch_5.h5')

2022-12-23 17:28:00.779292: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-23 17:28:00.779317: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ti-lap-492): /proc/driver/nvidia/version does not exist
2022-12-23 17:28:00.779537: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def pred_using_h5_digit(model, path, write_path, gt = None, prediction= None, wrong_results = None):
    result = {}
    wrong_results = {}
    gt=[]
    prediction = []
    wrong_count=0
    for img1 in sorted(glob.iglob(path)):
        image_name = img1.split('/')[-1]
        img=cv2.imread(img1)
        write_img = img
        img=cv2.resize(img,(28,28))
        img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.astype('float32') / 255.
        img= np.reshape(img,(1,28,28,1))
        res=model.predict(img)
        pred=res[0].argmax(axis=0)
        ground_truth = int(img1.split('/')[-2])
        gt.append(int(ground_truth))
        prediction.append(pred)
        result[img1] = pred
        if pred!= int(ground_truth):
            wrong_count+=1
            wrong_results[img1] = pred
            cv2.imwrite(write_path+image_name, write_img)
    accuracy = (len(glob.glob(path))-wrong_count)/len(glob.glob(path))
    return result, accuracy, gt, prediction, wrong_results

In [4]:
path = os.path.expanduser('~')+'/matching_grey_scale/0/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/0/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

2022-12-23 17:28:08.819827: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [5]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.9163069994370426
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      5329
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0

    accuracy                           0.92      5329
   macro avg       0.17      0.15      0.16      5329
weighted avg       1.00      0.92      0.96      5329



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
len(wrong_results)

225

In [8]:
wrong_results

{'/home/venkateshiyer/matching_grey_scale/0/hsf_0_00002.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00003.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00005.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00006.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00007.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00009.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00010.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00011.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00016.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00017.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00019.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00020.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00023.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00062.png': 1,
 '/home/venkateshiyer/matching_grey_scale/0/hsf_0_00065.png': 1,
 '/home/venkateshiyer/mat

In [17]:
path = os.path.expanduser('~')+'/matching_grey_scale/1/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/1/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

In [18]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       791

    accuracy                           1.00       791
   macro avg       1.00      1.00      1.00       791
weighted avg       1.00      1.00      1.00       791



In [19]:
len(wrong_results)

0

In [20]:
wrong_results

{}

In [21]:
path = os.path.expanduser('~')+'/matching_grey_scale/2/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/2/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

In [22]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.636986301369863
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.64      0.78       146
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.64       146
   macro avg       0.14      0.09      0.11       146
weighted avg       1.00      0.64      0.78       146



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
len(wrong_results)

53

In [24]:
wrong_results

{'/home/venkateshiyer/matching_grey_scale/2/hsf_0_00023.png': 1,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00030.png': 7,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00035.png': 1,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00220.png': 3,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00234.png': 1,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00291.png': 0,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00324.png': 3,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00327.png': 3,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00396.png': 1,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00441.png': 1,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00528.png': 3,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_00846.png': 0,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_01291.png': 0,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_01295.png': 0,
 '/home/venkateshiyer/matching_grey_scale/2/hsf_0_01510.png': 3,
 '/home/venkateshiyer/mat

In [25]:
path = os.path.expanduser('~')+'/matching_grey_scale/3/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/3/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

In [26]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.8644366197183099
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       1.00      0.86      0.93       568
           5       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0

    accuracy                           0.86       568
   macro avg       0.17      0.14      0.15       568
weighted avg       1.00      0.86      0.93       568



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
len(wrong_results)

77

In [28]:
wrong_results

{'/home/venkateshiyer/matching_grey_scale/3/hsf_0_00016.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00017.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00018.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00019.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00024.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00082.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00402.png': 0,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00472.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00595.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00596.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00597.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00602.png': 0,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00603.png': 1,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00605.png': 5,
 '/home/venkateshiyer/matching_grey_scale/3/hsf_0_00611.png': 1,
 '/home/venkateshiyer/mat

In [29]:
path = os.path.expanduser('~')+'/matching_grey_scale/4/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/4/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

In [30]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.20433890354734682
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           4       1.00      0.20      0.34      3411
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0

    accuracy                           0.20      3411
   macro avg       0.10      0.02      0.03      3411
weighted avg       1.00      0.20      0.34      3411



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
len(wrong_results)

2714

In [32]:
wrong_results

{'/home/venkateshiyer/matching_grey_scale/4/hsf_0_00002.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00004.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00006.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00007.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00008.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00009.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00010.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00012.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00013.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00018.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00025.png': 9,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00030.png': 9,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00032.png': 7,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00033.png': 1,
 '/home/venkateshiyer/matching_grey_scale/4/hsf_0_00034.png': 7,
 '/home/venkateshiyer/mat

In [33]:
path = os.path.expanduser('~')+'/matching_grey_scale/5/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/5/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

In [34]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.6872536136662286
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       1.00      0.69      0.81      1522
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0

    accuracy                           0.69      1522
   macro avg       0.10      0.07      0.08      1522
weighted avg       1.00      0.69      0.81      1522



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
len(wrong_results)

476

In [36]:
wrong_results

{'/home/venkateshiyer/matching_grey_scale/5/hsf_0_00009.png': 0,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00010.png': 1,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00017.png': 0,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00027.png': 6,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00031.png': 6,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00052.png': 7,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00057.png': 7,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00077.png': 1,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00079.png': 1,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00081.png': 1,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00082.png': 1,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00085.png': 1,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00086.png': 1,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00087.png': 1,
 '/home/venkateshiyer/matching_grey_scale/5/hsf_0_00162.png': 1,
 '/home/venkateshiyer/mat

In [37]:
path = os.path.expanduser('~')+'/matching_grey_scale/6/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/6/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

In [38]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.5817954332186425
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       1.00      0.58      0.74      3197
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0

    accuracy                           0.58      3197
   macro avg       0.10      0.06      0.07      3197
weighted avg       1.00      0.58      0.74      3197



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
len(wrong_results)

1337

In [40]:
wrong_results

{'/home/venkateshiyer/matching_grey_scale/6/hsf_0_00001.png': 1,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00002.png': 1,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00003.png': 0,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00006.png': 1,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00008.png': 1,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00009.png': 1,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00010.png': 1,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00011.png': 0,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00012.png': 0,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00014.png': 0,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00018.png': 0,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00020.png': 0,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00022.png': 0,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00028.png': 1,
 '/home/venkateshiyer/matching_grey_scale/6/hsf_0_00031.png': 0,
 '/home/venkateshiyer/mat

In [41]:
path = os.path.expanduser('~')+'/matching_grey_scale/7/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/7/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

In [42]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.8494208494208494
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           7       1.00      0.85      0.92       518
           9       0.00      0.00      0.00         0

    accuracy                           0.85       518
   macro avg       0.17      0.14      0.15       518
weighted avg       1.00      0.85      0.92       518



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
len(wrong_results)

78

In [44]:
wrong_results

{'/home/venkateshiyer/matching_grey_scale/7/hsf_0_00176.png': 2,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00356.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00357.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00389.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00393.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00479.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00666.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00718.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00777.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00778.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00796.png': 2,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_00935.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_01103.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_01352.png': 1,
 '/home/venkateshiyer/matching_grey_scale/7/hsf_0_01370.png': 1,
 '/home/venkateshiyer/mat

In [45]:
path = os.path.expanduser('~')+'/matching_grey_scale/8/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/8/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

In [47]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.3370665517984062
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       1.00      0.34      0.50      4643
           9       0.00      0.00      0.00         0

    accuracy                           0.34      4643
   macro avg       0.10      0.03      0.05      4643
weighted avg       1.00      0.34      0.50      4643



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
len(wrong_results)

3078

In [49]:
wrong_results

{'/home/venkateshiyer/matching_grey_scale/8/hsf_0_00000.png': 1,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00001.png': 1,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00002.png': 1,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00003.png': 1,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00004.png': 0,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00005.png': 7,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00006.png': 1,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00007.png': 1,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00008.png': 7,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00009.png': 0,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00010.png': 1,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00011.png': 1,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00012.png': 0,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00014.png': 1,
 '/home/venkateshiyer/matching_grey_scale/8/hsf_0_00015.png': 1,
 '/home/venkateshiyer/mat

In [50]:
path = os.path.expanduser('~')+'/matching_grey_scale/9/*'
write_path = os.path.expanduser('~')+'/matching_grey_scale_mc/9/'
result, accuracy, gt, prediction, wrong_results = pred_using_h5_digit(model,path, write_path)

In [51]:
print(accuracy)
from sklearn import metrics
print(metrics.classification_report(gt, prediction))

0.5409280500521376
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       1.00      0.54      0.70      3836
          10       0.00      0.00      0.00         0

    accuracy                           0.54      3836
   macro avg       0.10      0.05      0.07      3836
weighted avg       1.00      0.54      0.70      3836



/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/venkateshiyer/ocrtool/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
len(wrong_results)

1761

In [53]:
wrong_results

{'/home/venkateshiyer/matching_grey_scale/9/hsf_0_00000.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00001.png': 7,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00002.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00003.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00004.png': 7,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00005.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00006.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00007.png': 7,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00008.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00009.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00011.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00013.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00014.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00015.png': 1,
 '/home/venkateshiyer/matching_grey_scale/9/hsf_0_00016.png': 1,
 '/home/venkateshiyer/mat